<a href="https://colab.research.google.com/github/Naveen148/Naveen-M/blob/main/naveen_akaike_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

from nltk.corpus import stopwords
import string
import json
from nltk.sentiment.util import *
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from nltk.corpus import stopwords

In [15]:
stop_words = set(stopwords.words("english"))

In [16]:
with open(r'/content/tweets.json') as jfile:
   json_file = json.load(jfile)

In [18]:
df=pd.DataFrame(json_file).T

In [19]:
df.head()

,tweet_author,tweet_text
1374140386071961602,Hematopoiesis News,⚕️ Scientists conducted a Phase II study of ac...
1374032432173842437,"Michael Wang, MD",This phase 2 Acalabrutinib-Venetoclax (AV) tri...
1373902876553048065,1stOncology,#NICE backs #AstraZenecas #Calquence for #CLL ...
1373656782367813635,Toby Eyre,#acalabrutinib is a valuable option in pts int...
1372941634334232586,Lymphoma Hub,NICE has recommended the use of acalabrutinib ...


# **check null values

Total no of unique writer**

In [20]:
print('total no of null valus in the data:\n',df.isnull().sum())
print('total no tweet author:',df.tweet_author.nunique())

total no of null valus in the data:
 tweet_author    0
tweet_text      0
dtype: int64
total no tweet author: 9292


# **Clean the data and convert them into Machine Readable Code**

In [22]:
def clean_data(data):
  data = data.lower()
  data = re.sub(r'[^(a-zA-Z)\s]','', data)
     #     remove urls
  data = re.sub(r'http\S+', " ", data)
 #     remove mentions
  data = re.sub(r'@\w+',' ',data)
 #     remove hastags
  data = re.sub(r'#\w+', ' ',data)
#     remove digits
  data = re.sub(r'\d+()', ' ', data)
#     remove html tags and umber
  data = re.sub('r<.*?>',' ', data)
 #     remove stop words 
  data = data.split()
  data = " ".join([word for word in data if not word in stop_words])
  return data
df['tweet_text']=df['tweet_text'].apply(lambda x:clean_data(x))
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df['tweet_text']= df['tweet_text'].apply(lambda x: remove_punct(x))
#Tokenization of the text data
def tokenization(text):
    text = re.split('\W+', text)
    return text
df['tweet_text'] = df['tweet_text'].apply(lambda x: tokenization(x.lower()))

In [23]:
df1=df['tweet_text']

# **Steaming anf Lemitization of the code**

In [24]:
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text
df1 = df1.apply(lambda x: stemming(x))

In [30]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [26]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [28]:
from nltk.corpus import wordnet

In [31]:
wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
df1= df1.apply(lambda x: lemmatizer(x))

# **Stanford NLP NER**

In [32]:
from nltk.tag.stanford import StanfordNERTagger
!wget 'https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip'
!unzip stanford-ner-2018-10-16.zip

nltk.download('punkt')

st = StanfordNERTagger('/content/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/content/stanford-ner-2018-10-16/stanford-ner.jar',
                       encoding='utf-8')

--2022-11-11 11:08:53--  https://nlp.stanford.edu/software/stanford-ner-2018-10-16.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2018-10-16.zip [following]
--2022-11-11 11:08:53--  https://downloads.cs.stanford.edu/nlp/software/stanford-ner-2018-10-16.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180358328 (172M) [application/zip]
Saving to: ‘stanford-ner-2018-10-16.zip’

stanford-ner-2018-1 100%[===================>] 172.00M  4.45MB/s    in 30s     

2022-11-11 11:09:24 (5.67 MB/s) - ‘stanford-ner-2018-10-16.zip’ saved [180358328/180358328]

Archive:  stanford-ner-2018-10-16.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
val = [item if isinstance(df1, str) else " ".join(item) for item in df1 ]
seen = set()
val = [x for x in val if x not in seen and not seen.add(x)]

In [34]:
classified_text = st.tag(val)

In [40]:
entity= pd.DataFrame(classified_text,columns=['Entity Name','Entity Type'])

#Here We Remove Entities_type column from data we does not requierd this but the help to recgonized the type of entity

all_entities = (entity.groupby(by=['Entity Name'])
                          .size()
                          .sort_values(ascending=False)
                          .reset_index().rename(columns={0 : 'Frequency'}))
all_entities.head(10)

,Entity Name,Frequency
0,acalabrutinib,1306
1,calquenc,893
2,patient,790
3,covid,694
4,astrazeneca,598
5,cll,562
6,trial,425
7,lymphocyt,388
8,chronic,351
9,leukemia,342


In [42]:
all_entities.to_csv('objective1.csv')


In [36]:
df['tweet_text'] = df['tweet_text'].astype('str')
def get_polarity(text):
  return TextBlob(text).sentiment.polarity
df['Polarity'] = df['tweet_text'].apply(get_polarity)

In [43]:
df.to_csv('objective2.csv')

In [38]:
df.head()

,tweet_author,tweet_text,Polarity
1374140386071961602,Hematopoiesis News,"['scientists', 'conducted', 'phase', 'ii', 'st...",0.0
1374032432173842437,"Michael Wang, MD","['phase', 'acalabrutinibvenetoclax', 'av', 'tr...",0.0
1373902876553048065,1stOncology,"['nice', 'backs', 'astrazenecas', 'calquence',...",0.6
1373656782367813635,Toby Eyre,"['acalabrutinib', 'valuable', 'option', 'pts',...",0.1
1372941634334232586,Lymphoma Hub,"['nice', 'recommended', 'use', 'acalabrutinib'...",0.6
